In [ ]:
import os
import sys
import soundfile as sf
import torch
from tqdm import tqdm
from google.colab import files
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
files.upload()

Saving data_utils.py to data_utils (1).py


{'data_utils (1).py': b'# \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80 data_utils.py \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\r\nfrom __future__ import annotations\r\n\r\nimport os\r\nimport torch\r\nimport torchaudio\r\nimport pandas as pd\r\nfrom sklearn.model_selection import train_test_split\r\nfrom torch.utils.data import Dataset, DataLoader\r\n\r\n__all__ = ["DEAMHandler", "WaveDataset"]\r\n\r\n\r\n# \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\x

In [ ]:
from data_utils_old import DEAMHandler

handler = DEAMHandler()

Dataset root: /kaggle/input/deam-mediaeval-dataset-emotional-analysis-in-music
Audio directory: /kaggle/input/deam-mediaeval-dataset-emotional-analysis-in-music/DEAM_audio/MEMD_audio

Dataset Preview:
   song_id  valence_mean  valence_std  arousal_mean  arousal_std
0        2           3.1         0.94           3.0         0.63
1        3           3.5         1.75           3.3         1.62
2        4           5.7         1.42           5.5         1.63
3        5           4.4         2.01           5.3         1.85
4        7           5.8         1.47           6.4         1.69

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   song_id       1744 non-null   int64  
 1   valence_mean  1744 non-null   float64
 2   valence_std   1744 non-null   float64
 3   arousal_mean  1744 non-null   float64
 4   arousal_std   1744 non-nu

In [ ]:
wave10, sr = handler.get_waveform(10, verbose=True) # DEAM song #10
print(wave10.shape, sr)

Audio loaded: 720976 samples at 16000 Hz
(720976,) 16000


In [ ]:
CACHE_DIR = "wav_cache"
TARGET_SR = 16_000                  # <— pick whatever you like (16 k, 22 050 …)

present_ids  = sorted(handler.static_annotations["song_id"].tolist())

# ─── Probe once to find longest duration at the *chosen* SR ────────────────
lengths = []

print(f"🔎 Scanning {len(present_ids)} songs at {TARGET_SR} Hz …")
for sid in present_ids:
    wav, _ = handler.get_waveform(sid, target_sr=TARGET_SR, mono=True)
    lengths.append(wav.shape[-1])

MAX_LEN  = max(lengths)
MAX_SECS = MAX_LEN / TARGET_SR

print(f"✔ Common sample-rate : {TARGET_SR} Hz")
print(f"✔ Longest clip       : {MAX_LEN} samples ≈ {MAX_SECS:.2f} s\n")

# ─── Caching all IDs (present or missing) ──────────────────────────────────
os.makedirs(CACHE_DIR, exist_ok=True)

MAX_ID     = max(present_ids)
ID_WIDTH   = len(str(MAX_ID))

print(f"💾 Writing {MAX_ID} fixed-length tensors into {CACHE_DIR}/ …\n")

for idx, sid in enumerate(range(1, MAX_ID + 1), start=1):
    sid_str   = str(sid).zfill(ID_WIDTH)
    out_path  = os.path.join(CACHE_DIR, f"{sid_str}.pt")
    remaining = MAX_ID - idx

    if os.path.exists(out_path):
        print(f"[{idx}/{MAX_ID}] {sid_str}  ✔ already cached  (remaining: {remaining})")
        continue

    if sid in present_ids:
        wav, _ = handler.get_waveform(sid, target_sr=TARGET_SR, mono=True)
        wav = wav.squeeze(0)[:MAX_LEN]                    # 1-D
        if wav.numel() < MAX_LEN:
            wav = F.pad(wav, (0, MAX_LEN - wav.numel()))
        status = "✅ cached"
    else:
        wav = torch.zeros(MAX_LEN)
        status = "Ⓜ missing → silence"

    torch.save(wav.unsqueeze(0), out_path)                # [1, MAX_LEN]
    print(f"[{idx}/{MAX_ID}] {sid_str}  {status}  (remaining: {remaining})")

print("\n🎉 Done! All IDs now have uniform-SR tensors in wav_cache/")


🔎 Scanning 1744 songs at 16000 Hz …
✔ Common sample-rate : 16000 Hz
✔ Longest clip       : 726579 samples ≈ 45.41 s

💾 Writing 2000 fixed-length tensors into wav_cache/ …

[1/2000] 0001  Ⓜ missing → silence  (remaining: 1999)


ValueError: cannot select an axis to squeeze out which has size not equal to one

In [ ]:
!mv proc_cache/*.pt /content/drive/MyDrive/Colab\ Notebooks/DeepProject/DEAM_wav/

In [ ]:
AUDIO_DIR = "/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav"

# 3) Output file
csv_out = f"{AUDIO_DIR}/static_annotations.csv"

# 4) Save the DataFrame that’s already loaded in memory
#    (replace `old_handler` with whatever variable holds your first DEAMHandler)
handler.static_annotations.to_csv(csv_out, index=False)

print(f"✔ Saved static annotations to:\n   {csv_out}")

✔ Saved static annotations to:
   /content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav/static_annotations.csv


In [ ]:
from pathlib import Path
import shutil

ROOT  = Path("/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav")
SONGS = ROOT / "songs"
SONGS.mkdir(exist_ok=True)

# 2️⃣  Move any loose .pt files into DEAM_wav/songs/
for pt in ROOT.glob("*.pt"):
    target = SONGS / pt.name
    if not target.exists():               # don’t overwrite
        shutil.move(pt, target)
        print(f"→ moved {pt.name} to songs/")

# 3️⃣  Save the static annotations beside songs/
csv_out = ROOT / "annotations.csv"
handler.static_annotations.to_csv(csv_out, index=False)
print(f"\n✔ annotations.csv saved to {csv_out}")

print("\nFolder structure now:")
!tree -L 2 "{ROOT}"


✔ annotations.csv saved to /content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav/annotations.csv

Folder structure now:
/bin/bash: line 1: tree: command not found


In [ ]:
files.upload()

Saving data_utils.py to data_utils.py


{'data_utils.py': b'# \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80 data_utils.py \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\r\nfrom __future__ import annotations\r\n\r\nimport os\r\nimport torch\r\nimport torchaudio\r\nimport pandas as pd\r\nfrom sklearn.model_selection import train_test_split\r\nfrom torch.utils.data import Dataset, DataLoader\r\n\r\n__all__ = ["DEAMHandler", "WaveDataset"]\r\n\r\n\r\n# \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x

In [ ]:
# ╔══════════════════  REBUILD REAL-SONG CACHE  ═════════════════════╗
#   Downloads DEAM (data_utils_old) and saves *only* present IDs
# ╚══════════════════════════════════════════════════════════════════╝
import os, shutil, glob, torch, torch.nn.functional as F
from pathlib import Path
from tqdm.auto import tqdm
from data_utils_old import DEAMHandler      # Kaggle-downloading helper

# ── Paths ───────────────────────────────────────────────────────────
ROOT  = Path("/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav")
SONGS = ROOT / "songs"
SONGS.mkdir(parents=True, exist_ok=True)

# ── 1) Ensure annotations + fresh download ─────────────────────────
old_handler = DEAMHandler()                                   # downloads if needed
csv_src = old_handler.path / Path(old_handler._STATIC_CSV)
shutil.copy(csv_src, ROOT / "annotations.csv")
print("✔ annotations.csv copied")

# ── 2) Purge ALL existing tensors ─────────────────────────────────
removed = 0
for pt in glob.glob(str(SONGS / "*.pt")):
    os.remove(pt); removed += 1
print(f"🗑️  removed {removed} old .pt files")

# ── 3) Determine target SR + MAX_LEN (after resample) ─────────────
TARGET_SR   = 16_000
present_ids = sorted(old_handler.static_annotations["song_id"].tolist())

lengths = []
print(f"🔍 Scanning {len(present_ids)} songs @ {TARGET_SR} Hz …")
for sid in tqdm(present_ids, desc="probe"):
    wav, _ = old_handler.get_waveform(sid, target_sr=TARGET_SR, mono=True)
    lengths.append(wav.shape[-1])

MAX_LEN = max(lengths)
PAD     = len(str(max(present_ids)))            # zero-pad width
print(f"✔ MAX_LEN = {MAX_LEN} samples ≈ {MAX_LEN/TARGET_SR:.2f} s\n")

# ── 4) Cache ONLY real songs ───────────────────────────────────────
print(f"💾 Writing {len(present_ids)} tensors into {SONGS}/ …")
for sid in tqdm(present_ids, desc="cache"):
    fname   = SONGS / f"{sid:0{PAD}d}.pt"
    wav_np, _ = old_handler.get_waveform(sid, target_sr=TARGET_SR, mono=True)

    wav = torch.as_tensor(wav_np, dtype=torch.float32)[:MAX_LEN]  # 1-D
    if wav.numel() < MAX_LEN:                                     # pad end
        wav = F.pad(wav, (0, MAX_LEN - wav.numel()))

    torch.save(wav.unsqueeze(0), str(fname))                      # [1, MAX_LEN]

print(f"\n✅ Rebuild complete: {len(present_ids)} real-song tensors saved.")
# ═══════════════════════════════════════════════════════════════════


Dataset root: /kaggle/input/deam-mediaeval-dataset-emotional-analysis-in-music
Audio directory: /kaggle/input/deam-mediaeval-dataset-emotional-analysis-in-music/DEAM_audio/MEMD_audio

Dataset Preview:
   song_id  valence_mean  valence_std  arousal_mean  arousal_std
0        2           3.1         0.94           3.0         0.63
1        3           3.5         1.75           3.3         1.62
2        4           5.7         1.42           5.5         1.63
3        5           4.4         2.01           5.3         1.85
4        7           5.8         1.47           6.4         1.69

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   song_id       1744 non-null   int64  
 1   valence_mean  1744 non-null   float64
 2   valence_std   1744 non-null   float64
 3   arousal_mean  1744 non-null   float64
 4   arousal_std   1744 non-nu

probe:   0%|          | 0/1744 [00:00<?, ?it/s]

✔ MAX_LEN = 726579 samples ≈ 45.41 s

💾 Writing 1744 tensors into /content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav/songs/ …


cache:   0%|          | 0/1744 [00:00<?, ?it/s]


✅ Rebuild complete: 1744 real-song tensors saved.


In [ ]:
# Cell: Remove leading zeros from .pt filenames in DEAM_wav/songs/
from pathlib import Path
import os

# Adjust this path to your project location
SONGS_DIR = Path("/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav/songs")

renamed = 0
skipped = 0

for pt_file in SONGS_DIR.glob("*.pt"):
    stem = pt_file.stem  # filename without extension
    # Only process numeric stems that start with '0'
    if stem.isdigit() and stem.startswith("0"):
        raw_id = str(int(stem))           # "0079" → "79"
        new_name = f"{raw_id}.pt"
        new_path = SONGS_DIR / new_name

        if not new_path.exists():
            os.rename(pt_file, new_path)
            renamed += 1
        else:
            skipped += 1
    else:
        skipped += 1

print(f"✅ Renamed {renamed} files (removed padding).")
print(f"⏭️  Skipped {skipped} files (already raw-ID or conflict).")


✅ Renamed 743 files (removed padding).
⏭️  Skipped 1001 files (already raw-ID or conflict).


# Further  pre-process


## loading new data_utils

In [ ]:
from google.colab import files
files.upload()

Saving data_utils.py to data_utils.py


{'data_utils.py': b'# \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80 data_utils.py \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\r\nfrom __future__ import annotations\r\n\r\nimport os\r\nimport torch\r\nimport torchaudio\r\nimport pandas as pd\r\nfrom sklearn.model_selection import train_test_split\r\nfrom torch.utils.data import Dataset, DataLoader\r\n\r\n__all__ = ["DEAMHandler", "WaveDataset"]\r\n\r\n\r\n# \xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x94\x80\xe2\x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# precompute_wav2vec_feats.py  (fixed)
# -------------------------------------------------------------

import os, torch
from tqdm import tqdm
from transformers import AutoFeatureExtractor, Wav2Vec2Model
from data_utils import DEAMHandler

AUDIO_ROOT = "/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav"
MODEL_ID   = "ALM/wav2vec2-base-audioset"
OUT_DIR    = "/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_feats/songs"
BATCH      = 16                  # adjust to your GPU memory
DTYPE      = torch.float16       # or torch.float32

os.makedirs(OUT_DIR, exist_ok=True)
device   = "cuda" if torch.cuda.is_available() else "cpu"
encoder  = (
    Wav2Vec2Model
    .from_pretrained(MODEL_ID, output_hidden_states=True)
    .eval()
    .to(device)
)
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_ID)
for p in encoder.parameters():
    p.requires_grad_(False)

handler  = DEAMHandler(audio_root=AUDIO_ROOT)
song_ids = handler.static_annotations["song_id"].tolist()

def slices(lst, n):
    """Yield successive chunks of size n."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

for chunk in tqdm(list(slices(song_ids, BATCH)), desc="songs"):
    waves, done_ids = [], []
    for sid in chunk:
        out_path = os.path.join(OUT_DIR, f"{sid}.pt")
        if os.path.exists(out_path):
            continue
        wav, _ = handler.get_waveform(sid, target_sr=16_000, mono=True)
        waves.append(wav.squeeze().numpy())
        done_ids.append((sid, out_path))
    if not waves:
        continue

    inputs = feature_extractor(
        waves,
        sampling_rate=16_000,
        return_tensors="pt",
        padding=True,              # pad to longest in batch
        return_attention_mask=True
    )

    with torch.no_grad():
        # hidden_states is a tuple: [layer0, layer1, ..., layerN]
        # we want the output just *before* the last 2 layers:
        h_states = encoder(
            inputs.input_values.to(device)
        ).hidden_states[-3]         # shape [B, T_max, 768]

    frame_counts = inputs.attention_mask.sum(1)  # valid lengths per example

    # ── FIXED: iterate by batch index, slice time axis ──
    for i, ((sid, out_path), n) in enumerate(zip(done_ids, frame_counts)):
        # take the i-th example in the batch, then slice off padding
        single = h_states[i, : n].to("cpu", dtype=DTYPE).contiguous()  # [T_i, 768]
        torch.save(single, out_path)


✔ static annotations : /content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav/static_annotations.csv
✔ audio root         : /content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav/songs


songs: 100%|██████████| 109/109 [1:00:40<00:00, 33.40s/it]


## Arranging the data's folder

In [ ]:
from pathlib import Path
import shutil

FEAT_ROOT = Path("/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_feats")
WAV_ROOT  = Path("/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav")

SONG_DST  = FEAT_ROOT / "songs"
SONG_DST.mkdir(exist_ok=True)

#  move every *.pt
for pt_file in FEAT_ROOT.glob("*.pt"):
    shutil.move(pt_file, SONG_DST / pt_file.name)

# move the annotations CSV
ann_src = WAV_ROOT / "static_annotations.csv"
ann_dst = FEAT_ROOT / ann_src.name
if ann_src.exists():
    shutil.move(ann_src, ann_dst)
    print(f"moved {ann_src} → {ann_dst}")
else:
    print("⚠️  static_annotations.csv not found in DEAM_wav")

# confirm counts
print(f"{len(list((SONG_DST).glob('*.pt')))} feature files now in {SONG_DST}")


moved /content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_wav/static_annotations.csv → /content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_feats/static_annotations.csv
1744 feature files now in /content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_feats/songs


### quick check - can remove

In [ ]:
from pathlib import Path, PurePosixPath
import shutil, itertools

FEAT_ROOT = Path("/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_feats")
SONG_DST  = FEAT_ROOT / "songs"

parent_pts = list(FEAT_ROOT.glob("*.pt"))
child_pts  = list(SONG_DST.glob("*.pt"))

print(f"In parent: {len(parent_pts)}   in songs/: {len(child_pts)}")

# ── move any stragglers ───────────────────────────────────────────────
for f in parent_pts:
    target = SONG_DST / f.name
    if not target.exists():
        shutil.move(f, target)
        print("moved", f.name)


In parent: 0   in songs/: 1744


In [ ]:
import glob, os
list(glob.glob("/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_feats/**/static_annotations.csv",
               recursive=True))


['/content/drive/MyDrive/Colab Notebooks/DeepProject/DEAM_feats/static_annotations.csv']